## calculate the number of occurrences of temperature exceeding a hot or cold threshold in the nClimGrid-daily dataset; plot these on maps

### this requires having access to the nClimGrid-daily netcdf files, which ends up being a rather large dataset. They can be downloaded from https://www.ncei.noaa.gov/data/nclimgrid-daily/access/grids/ (this dataset is not provided here; you will need to download it to reproduce these figures).

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.dates as mdates 
import matplotlib.colors as mcolors
import numpy as np
import os

from metpy.plots import USCOUNTIES
from matplotlib.offsetbox import AnchoredText

from palettable.colorbrewer.diverging import RdBu_11, RdBu_11_r

#from dask.distributed import Client
#client=Client()
#client

### read in nclimgrid daily data, subset to Colorado

#### note that you will need to change the path to reflect the location of the netcdf files on your system

In [ ]:
var = 'tmax'

lonmin=-109.096667
lonmax=-102.0
latmin=36.95
latmax=41.05

data = xr.open_mfdataset("/data/rschumac/nclimgrid/daily/ncdd-*.nc")[var].sel(lat=slice(latmin,latmax),lon=slice(lonmin,lonmax)).chunk('auto')
#.chunk(dict(time=-1, lon=170, lat=98))


In [ ]:
data

### count frequency of exceedance of a threshold at each gridpoint

In [ ]:
thresh = 95.  ## in F
exceed = xr.where((1.8*data+32.) >= thresh, 1, 0).load()

exceed


### add up the number of exceedances over time, which yields a single map

In [ ]:
exceed_map = exceed.sum(dim='time')
exceed_map

### calculate the exceedances over different time periods, and the difference between those periods

In [ ]:
exceed_7100 = exceed.sel(time=slice('1971-01-01','2000-12-31'))
exceed_0120 = exceed.sel(time=slice('2001-01-01','2020-12-31'))
exceed_diff = ((exceed_0120.sum(dim='time')/20.) - (exceed_7100.sum(dim='time')/30.))

### nan out the places where no days above the threshold have occurred
exceed_diff = exceed_diff.where(exceed.sum(dim='time') > 0)


### make a map of this

In [ ]:
crs = ccrs.LambertConformal(central_longitude=-105.0, central_latitude=40.)

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1,projection=crs)

## Colorado versions
lonmin=-109.5
lonmax=-101.5
latmin=36.4
latmax=41.5
#lonmin=-109.046667
#lonmax=-102.046667
#latmin=37.0
#latmax=41.0

ax.set_extent([lonmin,lonmax,latmin,latmax])
#ax.add_feature(cfeature.LAND)
ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor="gray", linewidth=0.4)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)

if thresh >= 100:
    clevs = np.arange(-10,11,1)
else:
    clevs = np.arange(-20,22,2)

## trend
cf1 = ax.contourf(exceed_diff.lon, exceed_diff.lat, exceed_diff,
                  clevs,
                  extend='both',
                  cmap=RdBu_11_r.mpl_colormap,
                  transform=ccrs.PlateCarree())

ax.set_title("difference in average # of days per year with high temperature≥"+str(int(thresh))+"°F\n2001-2020 minus 1971-2000", 
             fontsize=12, fontweight='semibold')
cb1 = fig.colorbar(cf1, ax=ax, orientation='horizontal', aspect=30, shrink=0.65, pad=0.01)
cb1.set_label('number of days', size='large')

# Add a text annotation 
text = AnchoredText("data source: NOAA/NCEI nClimGrid-daily",
                    loc='lower left', prop={'size': 8.15}, frameon=True)
ax.add_artist(text)

fig.savefig("diff_"+str(int(thresh))+"degdays.png",
           dpi=300,transparent=False, facecolor='white', bbox_inches='tight')

plt.show()


### now repeat for cold

#### again note that you will need to change the path to reflect the location of the netcdf files on your system

In [ ]:
var = 'tmin'

lonmin=-109.096667
lonmax=-102.0
latmin=36.95
latmax=41.05

data = xr.open_mfdataset("/data/rschumac/nclimgrid/daily/ncdd-*.nc")[var].sel(lat=slice(latmin,latmax),lon=slice(lonmin,lonmax)).chunk('auto')
#.chunk(dict(time=-1, lon=170, lat=98))

In [ ]:
thresh = 0.  ## in F
exceed = xr.where((1.8*data+32.) <= thresh, 1, 0).load()

exceed

In [ ]:
exceed_map = exceed.sum(dim='time')

exceed_7100 = exceed.sel(time=slice('1971-01-01','2000-12-31'))
exceed_0120 = exceed.sel(time=slice('2001-01-01','2020-12-31'))
exceed_diff = ((exceed_0120.sum(dim='time')/20.) - (exceed_7100.sum(dim='time')/30.))

### nan out the places where no days above the threshold have occurred
exceed_diff = exceed_diff.where(exceed.sum(dim='time') > 0)


In [ ]:
crs = ccrs.LambertConformal(central_longitude=-105.0, central_latitude=40.)

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1,projection=crs)

## Colorado versions
lonmin=-109.5
lonmax=-101.5
latmin=36.4
latmax=41.5
#lonmin=-109.046667
#lonmax=-102.046667
#latmin=37.0
#latmax=41.0

ax.set_extent([lonmin,lonmax,latmin,latmax])
#ax.add_feature(cfeature.LAND)
ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor="gray", linewidth=0.4)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)

#if thresh <= -10:
#    clevs = np.arange(-5,6,1)
#else:
#    clevs = np.arange(-10,11,1)
clevs = np.arange(-20,22,2)

## trend
cf1 = ax.contourf(exceed_diff.lon, exceed_diff.lat, exceed_diff,
                  clevs,
                  extend='both',
                  cmap=RdBu_11.mpl_colormap,
                  transform=ccrs.PlateCarree())

ax.set_title("difference in average # of days per year with low temperature≤"+str(int(thresh))+"°F\n2001-2020 minus 1971-2000", 
             fontsize=12, fontweight='semibold')
cb1 = fig.colorbar(cf1, ax=ax, orientation='horizontal', aspect=30, shrink=0.65, pad=0.01)
cb1.set_label('number of days', size='large')

# Add a text annotation 
text = AnchoredText("data source: NOAA/NCEI nClimGrid-daily",
                    loc='lower left', prop={'size': 8.15}, frameon=True)
ax.add_artist(text)

fig.savefig("diff_"+str(int(thresh))+"degdays.png",
           dpi=250,transparent=False, facecolor='white', bbox_inches='tight')

plt.show()
